# Topic Modeling con BERTopic

En este tutorial exploraremos cómo usar BERTopic para crear temas a partir del conocido conjunto de datos 20Newsgroups. Discutiremos los casos de uso y los métodos más frecuentes, junto con los parámetros importantes a tener en cuenta.

## BERTopic
BERTopic es una técnica de modelado de temas que aprovecha los transformers de 🤗 y una TF-IDF basada en clases personalizada para crear grupos densos que permiten obtener temas fácilmente interpretables y al mismo tiempo conservar palabras importantes en las descripciones de los temas.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Habilitar la GPU

Primero, debes habilitar las GPU para el cuaderno:

- Navega a Editar→Configuración del cuaderno
- selecciona GPU en el menú desplegable Acelerador de hardware

[Referencia](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Instalando BERTopic**

Comenzamos instalando BERTopic desde PyPi (para ejecutar en COLAB)

In [ ]:
%%capture
!pip install bertopic

## Reiniciar el cuaderno
Después de instalar BERTopic, algunos paquetes que ya estaban cargados fueron actualizados y para usarlos correctamente, ahora debemos reiniciar el cuaderno.

Desde el menú:

Entorno de ejecución → Reiniciar entorno de ejecución

# Datos
Para este ejemplo, utilizaremos el popular conjunto de datos "20 Newsgroups", que contiene aproximadamente 18000 publicaciones de grupos de noticias:
```
from sklearn.datasets import fetch_20newsgroups

docs = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))['data']
```
(hemos cambiado los datos al conjunto de datos Lee en Gensim)

In [ ]:
import requests

lee_data_url = 'https://www.dropbox.com/s/55pbyhi50gbvgtj/lee_background.cor?dl=1'


response = requests.get(lee_data_url)
data = response.text

docs = [d for d in data.split('\n')]
len(docs)

In [ ]:
docs[0]

# **Topic Modeling**

En este ejemplo, repasaremos los componentes principales de BERTopic y los pasos necesarios para crear un modelo de temas sólido.

## Entrenamiento

Comenzamos instanciando BERTopic. Establecemos el idioma en `english` ya que nuestros documentos están en inglés. Si deseas utilizar un modelo multilingüe, por favor usa `language="multilingual"` en su lugar.

También calcularemos las probabilidades de los temas. Sin embargo, esto puede ralentizar significativamente BERTopic cuando se trabaja con grandes cantidades de datos (>100_000 documentos). Se recomienda desactivarlo si deseas acelerar el modelo.

**NOTA**: Usa `language="multilingual"` para seleccionar un modelo que soporta 50+ lenguajes.

In [ ]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

topic_model = BERTopic(language="english", calculate_probabilities=True, verbose=True)

topics, probs = topic_model.fit_transform(docs)
#entrenamos y luego reducimos vocabulario
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2), min_df=0.5)
topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

BERTopic calcula internamente la matriz TF-IDF de cada cluster para determinar los términos más importantes.

In [ ]:
topic_model.c_tf_idf_

In [ ]:
#los términos de la matriz tf_idf están en topic_model.vectorizer_model.get_feature_names_out()
import numpy as np

np.random.choice(topic_model.vectorizer_model.get_feature_names_out(), 5, replace=False)

In [ ]:
topic_model.vectorizer_model

In [ ]:
topic_model.topic_labels_

## Extracción de temas
Después de ajustar nuestro modelo, podemos comenzar a ver los resultados. Por lo general, comenzamos por examinar los temas más frecuentes, ya que representan mejor la colección de documentos.

In [ ]:
freq = topic_model.get_topic_info(); freq.head(5)

El número -1 se refiere a todos los valores atípicos y generalmente se deben ignorar. A continuación, echemos un vistazo a un tema frecuente que se generó:

In [ ]:
topic_model.get_topic(0)  # Select the most frequent topic

**NOTA**: BERTopic es estocástico, lo que significa que los temas pueden variar en diferentes ejecuciones. Esto se debe principalmente a la naturaleza estocástica de UMAP.

## Atributos

Existen varios atributos a los que puedes acceder después de entrenar tu modelo de BERTopic:


| Atributo | Descripción |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | Los temas que se generan para cada documento después de entrenar o actualizar el modelo de temas. |
| probabilities_ | Las probabilidades que se generan para cada documento si se utiliza HDBSCAN. |
| topic_sizes_           | El tamaño de cada tema.                                                                      |
| topic_mapper_          | Una clase para realizar un seguimiento de los temas y sus mapeos cada vez que se fusionan/reducen.             |
| topic_representations_ | Las mejores *n* palabras por tema y sus respectivos valores c-TF-IDF.                             |
| c_tf_idf_              | La matriz de tema-término calculada mediante c-TF-IDF.                                       |
| topic_labels_          | Las etiquetas predeterminadas para cada tema.                                                          |
| custom_labels_         | Etiquetas personalizadas para cada tema generadas mediante `.set_topic_labels`.                                                               |
| topic_embeddings_      | Las incrustaciones para cada tema si se utilizó `embedding_model`.                                                              |
| representative_docs_   | Los documentos representativos para cada tema si se utiliza HDBSCAN.                                                |

Por ejemplo, para acceder a los temas predichos para los primeros 10 documentos, simplemente ejecutamos lo siguiente:

In [ ]:
topic_model.topics_[:10]

In [ ]:
topic_model.topic_sizes_

In [ ]:
topic_model.topic_labels_

# **Visualización**
Existen varias opciones de visualización disponibles en BERTopic, como la visualización de temas, probabilidades y temas a lo largo del tiempo. El modelado de temas es, hasta cierto punto, bastante subjetivo. Las visualizaciones ayudan a comprender los temas que se crearon.

## Visualizar temas
Después de haber entrenado nuestro modelo `BERTopic`, podemos iterativamente revisar quizás cientos de temas para tener una buena comprensión de los temas que se extrajeron. Sin embargo, esto lleva bastante tiempo y carece de una representación global. En cambio, podemos visualizar los temas que se generaron de manera muy similar a [LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualizar las probabilidades de los temas

La variable `probabilities` que se devuelve de `transform()` o `fit_transform()` se puede utilizar para comprender qué tan seguro está BERTopic de que ciertos temas se encuentren en un documento.

Para visualizar las distribuciones, simplemente llamamos a:

In [ ]:
topic_model.visualize_distribution(probs[1], min_probability=0.015)

## Visualizar la jerarquía de temas

Los temas que se crearon pueden ser reducidos jerárquicamente. Para comprender la estructura jerárquica potencial de los temas, podemos utilizar `scipy.cluster.hierarchy` para crear grupos y visualizar cómo se relacionan entre sí. Esto puede ayudar a seleccionar un número apropiado de temas al reducir la cantidad de temas que has creado.

In [ ]:
topic_model.visualize_hierarchy(top_n_topics=50)

## Visualizar términos

Podemos visualizar los términos seleccionados para algunos temas mediante la creación de gráficos de barras a partir de los puntajes c-TF-IDF para cada representación de tema. Podemos obtener información de los puntajes relativos de c-TF-IDF entre y dentro de los temas. Además, puedes comparar fácilmente las representaciones de los temas entre sí.

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 1), min_df=0.05)
topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

In [ ]:
topic_model.c_tf_idf_

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)

## Visualizar la similitud de temas

Habiendo generado incrustaciones de temas, tanto a través de c-TF-IDF como de incrustaciones, podemos crear una matriz de similitud simplemente aplicando similitudes coseno a través de esas incrustaciones de temas. El resultado será una matriz que indica qué tan similares son ciertos temas entre sí.

In [ ]:
topic_model.visualize_heatmap(n_clusters=5, width=500, height=500)

## Visualizar la disminución del puntaje de términos

Los temas están representados por una serie de palabras que comienzan con la mejor palabra representativa. Cada palabra está representada por un puntaje c-TF-IDF. Cuanto mayor sea el puntaje, más representativa será la palabra para el tema. Dado que las palabras del tema están ordenadas según su puntaje c-TF-IDF, los puntajes disminuyen lentamente a medida que se agregan palabras. En algún momento, agregar palabras a la representación del tema solo aumenta marginalmente el puntaje total de c-TF-IDF y no sería beneficioso para su representación.

Para visualizar este efecto, podemos trazar los puntajes c-TF-IDF para cada tema según el rango de términos de cada palabra. En otras palabras, la posición de las palabras (rango de términos), donde las palabras con el puntaje c-TF-IDF más alto tendrán un rango de 1, se colocarán en el eje x. Mientras que el eje y estará poblado por los puntajes c-TF-IDF. El resultado es una visualización que muestra la disminución del puntaje c-TF-IDF al agregar palabras a la representación del tema. Esto te permite, utilizando el método del codo, seleccionar el mejor número de palabras en un tema.

In [ ]:
topic_model.visualize_term_rank()

# **Representación de temas**
Después de haber creado el modelo de temas, es posible que no estés satisfecho con algunos de los parámetros que has elegido. Afortunadamente, BERTopic te permite actualizar los temas después de haber sido creados.

Esto te permite ajustar el modelo según tus especificaciones y deseos.

## Actualizar temas
Cuando hayas entrenado un modelo y hayas visto los temas y las palabras que los representan, es posible que no estés satisfecho con la representación. Tal vez olvidaste eliminar las palabras vacías (stopwords) o quieres probar un rango de n-gramas diferente. Podemos usar la función `update_topics` para actualizar la representación de los temas con nuevos parámetros para `c-TF-IDF`:

In [ ]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2), max_df=0.7)
topic_model.update_topics(docs, vectorizer_model=vectorizer_model)


In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

In [ ]:
topic_model.c_tf_idf_

## Reducción de temas
También podemos reducir el número de temas después de haber entrenado un modelo de BERTopic. La ventaja de hacer esto es que puedes decidir el número de temas después de saber cuántos se crean en realidad. Es difícil predecir antes de entrenar tu modelo cuántos temas hay en tus documentos y cuántos se extraerán. En cambio, podemos decidir después cuántos temas parecen realistas:

In [ ]:
topic_model.reduce_topics(docs, nr_topics=60)

In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

# **Buscar temas**
Después de haber entrenado nuestro modelo, podemos usar `find_topics` para buscar temas que sean similares a un término de búsqueda de entrada. Aquí, vamos a buscar temas que se relacionen estrechamente con el término de búsqueda "vehículo". Luego, extraemos el tema más similar y verificamos los resultados:

In [ ]:
similar_topics, similarity = topic_model.find_topics("fire", top_n=5); similar_topics

In [ ]:
topic_model.get_topic(7)

# **Modelos de embeddings**
El parámetro `embedding_model` recibe una cadena que apunta a un modelo de Sentence-Transformers, un SentenceTransformer o un modelo de embeddings de documentos de Flair.

## Sentence-Transformers
Puedes seleccionar cualquier modelo de sentence-transformers aquí y pasarlo a través de BERTopic con `embedding_model`:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

O selecciona un modelo de SentenceTransformer con tus propios parámetros:

In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Haz clic [aquí](https://www.sbert.net/docs/pretrained_models.html) para obtener una lista de los modelos de sentence transformers admitidos.

In [ ]:
topics, probs = topic_model.fit_transform(docs)


In [ ]:
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1, 2), max_df=0.7)
topic_model.update_topics(docs, vectorizer_model=vectorizer_model)

In [ ]:
topic_model.get_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=5)